In [0]:
!pip install pytorch-pretrained-bert
# !pip install pandas
# !pip install torch
# !pip install nltk
# !pip install tqdm
# !pip install seaborn
# !pip install numpy
# !pip install sklearn
import torch
import torch
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import re
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import files, drive
drive.mount('/content/gdrive')
# torch.save(encoder1.state_dict(), "gdrive/My Drive/CompLing/rusdeu10_beam_encoder1.dic")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
# logging.basicConfig(level=logging.INFO)

bertpath = 'gdrive/My Drive/CompLing/rubert'
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(bertpath, do_lower_case=False)
modelbert = BertModel.from_pretrained(bertpath)
modelbert.cuda()



def get_embedding(text):       
    marked_text = "[CLS] " + text + " [SEP]"
    # Tokenize our sentence with the BERT tokenizer.
    tokenized_text = tokenizer.tokenize(marked_text)
    # Print out the tokens.
    # print (tokenized_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    
    # Display the words with their indeces.
    # for tup in zip(tokenized_text, indexed_tokens):
    #     print('{:<12} {:>6,}'.format(tup[0], tup[1]))
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    tokens_tensor.cuda()
    segments_tensors.cuda()
    # print(tokens_tensor)
    # Put the model in "evaluation" mode, meaning feed-forward operation.
    modelbert.eval()
    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = modelbert(tokens_tensor, segments_tensors)   
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []
    # `token_embeddings` is a [22 x 12 x 768] tensor.
    token_embeddings = torch.stack(encoded_layers, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1, 0, 2)
#     print(token_embeddings.shape)
    # For each token in the sentence...
    for token in token_embeddings:
        # `token` is a [12 x 768] tensor
        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)    
    # Use `cat_vec` to represent `token`.
    token_vecs_cat.append(cat_vec)
    # print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))    
    return torch.stack(token_vecs_cat, dim=0)


In [0]:
# the_sentence = "Слон съел банан."
# the_bert_emb = get_embedding(the_sentence)
# print(the_bert_emb.shape)

In [0]:
train = pd.read_csv('gdrive/My Drive/CompLing/mailru_train.csv')
test = pd.read_csv('gdrive/My Drive/CompLing/mailru_test.csv')

In [0]:
puncts  = str.maketrans('','',":,?;\"!.«»()\'\*\+\\\/\>\<\=")
def smart_process_text(b):
    b = re.sub(r"(?<=[А-ЯЁа-яё\dA-Za-z])(?=\))", " ", b)    
    eyes, noses, mouths = r":;8BX=", r"-~'^", r")(/\|DP"
    patsmileys = "[%s][%s]?[%s]" % tuple(map(re.escape, [eyes, noses, mouths]))
    j = re.findall(patsmileys, b)
    patsmile = r"\s\)+"
    h = re.findall(patsmile, b)
#     b = re.sub(r"\?", " СПРАШИВАЮ ", b)
#     b = re.sub(r"\!", " ВАЖНО ", b)
    if j:
        b = re.sub(patsmileys, " УЛЫБАЮСЬ ", b)
    # print (j, b)
    if h and b.count('(') == 0:
        b = re.sub(patsmile, " УЛЫБАЮСЬ ", b)
    
    b = re.sub(r"о_о|o_o", " УДИВЛЯЮСЬ и УЛЫБАЮСЬ ", b, flags=re.IGNORECASE)
    b = re.sub(r"\)+\++", " ШУЧУ ", b)    
    b = re.sub(r"\)\)\)*", " СМЕЮСЬ ", b)
    b = re.sub(r"\(\(\(*", " ПОМОГИТЕ ", b)    
    b  = re.sub(r"\?", " СПРАШИВАЮ ", b)
    b  = re.sub(r"\$", " ДОЛЛАР ", b)
    b  = re.sub(r"\)\=", " МАТЕМАТИКА ", b)

    # specific
    b  = re.sub(r"✿‿✿", " ШУТКА ЮМОРА ", b) 
    b  = re.sub(r"\(\+\)", " ВОПРОС ", b) 
    b  = re.sub(r"хорар", " гороскоп ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"плез", " очень пожалуйста ", b, flags=re.IGNORECASE)     
    b  = re.sub(r"ryzen|райзен.*", " процессор ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"rtx", " видеокарта ", b, flags=re.IGNORECASE)     
    b  = re.sub(r"тдв", " темы для взрослых ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"безобманство", " глупая религия ", b, flags=re.IGNORECASE)     
    b  = re.sub(r"PascalABC.*", " плохой язык программирования ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"python.*", " хороший язык программирования ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"крмп|п[ау]бг.*|pubg|фортнайт.*|fortnite|roblox|роблокс.*|discord|дискорд.*", " компьютерная игра ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"зеленск[огмийу]+", " президент Украины ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"дошик.*", " плохие макароны ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"eлeна", " Елена любительница астрологии и магии ", b, flags=re.IGNORECASE) 
    b  = re.sub(r"☼\)\s+☼\)\s+☼\)", " загадка ", b)
    b  = re.sub(r"Девчин", " любовь ", b) # https://otvet.mail.ru/profile/id51163161/
    b = re.sub(r"\!", " ВАЖНО ", b)
    b = re.sub(r"\+\+\+*", " СОГЛАШАЮСЬ ", b)
    b = re.sub(r"%", " процентов ", b)
    b = re.sub(r"°\s*[CС]", " градусов Цельсия ", b)
    b = re.sub(r"°", " градусов ", b)
    b = re.sub(r"\"", "", b)
    
    words = wordpunct_tokenize(b.lower())    
    return words

In [0]:
word2freq = {}

for question in tqdm(train.question):    
    words = smart_process_text(question)    
    for word in words:        
        if word in word2freq:
            word2freq[word] += 1
        else:
            word2freq[word] = 1


100%|██████████| 666666/666666 [00:55<00:00, 12035.35it/s]

In [0]:
word2index = {'PAD': 0}
vectors = []
    
word2vec_file = open('gdrive/My Drive/CompLing/cc.ru.300.vec')
    
n_words, embedding_dim = word2vec_file.readline().split()
n_words, embedding_dim = int(n_words), int(embedding_dim)

# Zero vector for PAD
vectors.append(np.zeros((1, embedding_dim)))

progress_bar = tqdm(desc='Read word2vec', total=n_words)

while True:

    line = word2vec_file.readline().strip()

    if not line:
        break
        
    current_parts = line.split()
    current_word = ' '.join(current_parts[:-embedding_dim])

    if current_word in word2freq:
        word2index[current_word] = len(word2index)
        current_vectors = current_parts[-embedding_dim:]
        current_vectors = np.array(list(map(float, current_vectors)))
        current_vectors = np.expand_dims(current_vectors, 0)
        vectors.append(current_vectors)

    progress_bar.update(1)

progress_bar.close()
word2vec_file.close()
vectors = np.concatenate(vectors)

Read word2vec: 100%|██████████| 2000000/2000000 [02:00<00:00, 16636.06it/s]


In [0]:
unk_words = [word for word in word2freq if word not in word2index]
unk_counts = [word2freq[word] for word in unk_words]
n_unk = sum(unk_counts) * 100 / sum(list(word2freq.values()))

sub_sample_unk_words = {word: word2freq[word] for word in unk_words}
sorted_unk_words = list(sorted(sub_sample_unk_words, key=lambda x: sub_sample_unk_words[x], reverse=True))

print('Мы не знаем {:.2f} % слов в датасете'.format(n_unk))
print('Количество неизвестных слов {} из {}, то есть {:.2f} % уникальных слов в словаре'.format(
    len(unk_words), len(word2freq), len(unk_words) * 100 / len(word2freq)))
print('В среднем каждое встречается {:.2f} раз'.format(np.mean(unk_counts)))
print()
print('Топ 5 невошедших слов:')

for i in range(5):
    print(sorted_unk_words[i], 'с количеством вхождениий -', word2freq[sorted_unk_words[i]])

Мы не знаем 1.94 % слов в датасете
Количество неизвестных слов 106532 из 327438, то есть 32.54 % уникальных слов в словаре
В среднем каждое встречается 1.37 раз

Топ 5 невошедших слов:
). с количеством вхождениий - 2038
), с количеством вхождениий - 2027
.) с количеством вхождениий - 756
», с количеством вхождениий - 314
*** с количеством вхождениий - 280


In [0]:
class WordData(Dataset):
    
    def __init__(self, x_data, y_data, word2index, sequence_length=32, pad_token='PAD', verbose=True):
        
        super().__init__()
        
        self.x_data = []
        self.y_data = []
        self.b_data = []
        
        self.word2index = word2index
        self.sequence_length = sequence_length
        
        self.pad_token = pad_token
        self.pad_index = self.word2index[self.pad_token]
        
        self.load(x_data, y_data, verbose=verbose)
        
    @staticmethod
    def process_text(text):
        
        # Место для вашей предобработки
        words = smart_process_text(text)
#         words = wordpunct_tokenize(text.lower())

        return words
        
    def load(self, data, data2, verbose=True):
        
#         data_iterator = tqdm(data, desc='Loading data', disable=not verbose)
        
#         for text in data_iterator:
#             words = self.process_text(text)
#             indexed_words = self.indexing(words)
#             print(indexed_words)
#             self.x_data.append(indexed_words)
        
        data_iterator = tqdm(range(len(data)), desc='Loading data', disable=not verbose, position=0)

#         for i in data_iterator:
#             words = self.process_text(data[i])
#             emb = get_embedding(' '.join(words))
#             self.x_data.append(emb)
#             self.y_data.append(data2[i])
        
        
        for i in data_iterator:            
            words = self.process_text(data[i])
            indexed_words = self.indexing(words)
            self.x_data.append(indexed_words)
            self.y_data.append(data2[i])        
            
    
    def indexing(self, tokenized_text):

        # здесь мы не используем токен UNK, потому что мы мы его специально не учили
        # становится непонятно какой же эмбеддинг присвоить неизвестному слову,
        # поэтому просто выбрасываем наши неизветсные слова
        
        ### CODE ###

        return [self.word2index[token] if token in self.word2index else self.word2index["непонятно"] for token in tokenized_text]
    
    def padding(self, sequence):
        
        # Ограничить длину self.sequence_length
        # если длина меньше максимально - западить
        
        ### CODE ###

        if len(sequence) > self.sequence_length:
            sequence = sequence[:self.sequence_length]
        elif len(sequence) < self.sequence_length:
            sequence = sequence + [self.pad_index] * (self.sequence_length - len(sequence))

        return sequence
    
    def __len__(self):
        
        return len(self.x_data)
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx]
        x = self.padding(x)
        x = torch.Tensor(x).long()
        
        y = self.y_data[idx]
        
        return x, y

In [0]:
x_train, x_validation, y_train, y_validation = train_test_split(train.question, train.main_category, test_size=0.1)

train_dataset = WordData(list(x_train), list(y_train), word2index)
train_loader = DataLoader(train_dataset, batch_size=64)

validation_dataset = WordData(list(x_validation), list(y_validation), word2index)
validation_loader = DataLoader(validation_dataset, batch_size=64)

test_dataset = WordData(list(test.question), np.zeros((test.shape[0])), word2index)
test_loader = DataLoader(test_dataset, batch_size=64)

Loading data: 100%|██████████| 200000/200000 [00:16<00:00, 11787.79it/s]


In [0]:
for x, y in test_loader:
    break

In [0]:
x.shape

torch.Size([64, 32])

In [0]:
y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [0]:
n_classes = train.main_category.unique().shape[0]

In [0]:
# torch.cuda.empty_cache()
print("CUDA RAM usage", torch.cuda.max_memory_allocated(device))

def tsort(ts):
  z = torch.tensor([len(torch.nonzero(ts[i])) for i in range(ts.shape[0])]).sort(descending=True)
  return ts[z[1]], z[0], np.argsort(z[1]).to(device)

class DeepAverageNetwork(torch.nn.Module):
    
    def __init__(self, embedding_matrix, n_classes):
        
        super().__init__()
        
        self.embedding_layer = torch.nn.Embedding.from_pretrained(torch.Tensor(embedding_matrix))
        
        self.l1 = torch.nn.Linear(300, 256)
        self.lstm = torch.nn.LSTM(input_size=300,hidden_size=256*2,bidirectional=True, batch_first=True)
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(256, 128)
        self.lout = torch.nn.Linear(128, n_classes)
        
        self.embedding_length = 300
#         self.hidden_size = 64
        self.hidden_size = 64
        self.drop = nn.Dropout2d(0.25)    
        
        self.lstm = nn.LSTM(input_size=self.embedding_length,hidden_size=self.hidden_size,\
                        bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*2, n_classes)   
        
#         self.linear_attn = nn.Linear(self.hidden_size, n_classes)   
        self.linear_attn = nn.Linear(156, n_classes)   
    
        self.linear_cnn = nn.Linear(512, n_classes)   
        
        self.ak = nn.Linear(self.hidden_size*2, n_classes)
        self.aq = nn.Linear(self.hidden_size*2, n_classes)
        self.av = nn.Linear(self.hidden_size*2, n_classes)
        
        kerns = [2,4,6,8]
        seqlen = 32
        con_size = 64
        con_out = 64 * 2
        
        self.conv1 = nn.Conv1d(in_channels=con_size, out_channels= con_out, kernel_size=kerns[0])
        self.conv2 = nn.Conv1d(in_channels=con_size, out_channels= con_out, kernel_size=kerns[1])
        self.conv3 = nn.Conv1d(in_channels=con_size, out_channels= con_out, kernel_size=kerns[2])
        self.conv4 = nn.Conv1d(in_channels=con_size, out_channels= con_out, kernel_size=kerns[3])

        self.mp1 = nn.MaxPool1d(kernel_size=(seqlen-kerns[0]+1))
        self.mp2 = nn.MaxPool1d(kernel_size=(seqlen-kerns[1]+1))
        self.mp3 = nn.MaxPool1d(kernel_size=(seqlen-kerns[2]+1))
        self.mp4 = nn.MaxPool1d(kernel_size=(seqlen-kerns[3]+1))
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
#         print("in", x.shape)
        x_sorted, lens, unindex = tsort(x)
        x = self.embedding_layer(x_sorted)
#         print("with embs", x.shape)
        x = self.drop(x)
        z = pack_padded_sequence(x, lens, batch_first=True)
        x, _ =  self.lstm(z)                
        x, _ = pad_packed_sequence(x, batch_first=True)        
        x = x.index_select(0, unindex)
#         print("packing out", x.shape)
        x = x.transpose(0, 1)
#         print("transposed", x.shape)
        x = x[0]
#         print("one dim", x.shape)
#         x_key = self.attn_key(x)
#         x_que = self.attn_query(x)
#         x_val = self.attn_val(x)
        
#         att_scores = torch.bmm(x_que, x_key.transpose(1, 2))
#         print(att_scores.shape)
#         att_scores_scaled = (att_scores) / math.sqrt(x_key.shape[-1])
#         att_distribution = torch.nn.functional.softmax(att_scores_scaled, 2)
#         att_vectors = torch.bmm(att_distribution, x_val)
        lstm_out = x
        out_key = self.ak(lstm_out)
        out_query = self.aq(lstm_out)
        out_value = self.av(lstm_out)
        
        ok2 = out_key.transpose(0, 1)
        ov2 = out_value.transpose(0, 1)
#         print(out_query.shape, out_key.shape, ok2.shape)
        
#         att_scores = torch.bmm(out_query, out_key.transpose(1, 2))
        att_scores = torch.mm(out_query, ok2) 
#         print(att_scores.shape)
        sq = math.sqrt(out_key.shape[-1])
#         print("att_scores", att_scores.shape)
#         print("sq",sq)
        att_scores_scaled = (att_scores) / sq
#         print(att_scores_scaled.shape)
        
        att_distribution = torch.nn.functional.softmax(att_scores_scaled, 1)
#         print(att_distribution.shape)
        att_vectors = torch.mm(att_distribution, out_value)
#         print("vecs", att_vectors.shape)
        lstm_out_cnn = torch.cat([lstm_out, att_vectors], dim=-1)
#         print("cat", lstm_out_cnn.shape)
#         print(x.shape)
        x = lstm_out_cnn
#         x = x.transpose(0, 1)        
#         print(lstm_out_cnn.shape)
        x = self.drop(x)
        
#         x_transposed = x.transpose(1, 2)  
#         x_transposed = x.unsqueeze(0)
# #         print(x_transposed.shape)
#         x1 = F.relu(self.conv1(x_transposed))
#         x2 = F.relu(self.conv2(x_transposed))
#         x3 = F.relu(self.conv3(x_transposed))
#         x4 = F.relu(self.conv4(x_transposed))          

#         mp1x = self.mp1(x1)
#         mp2x = self.mp2(x2)
#         mp3x = self.mp3(x3)
#         mp4x = self.mp4(x4)
        
#         x_cat = torch.cat((mp1x, mp2x, mp3x, mp4x), 1)
                
#         print(x_cat.shape)
#         x = x_cat.squeeze(0)
#         print(x.shape)
#         x = x.transpose(0, 1)
        
#         print(x.shape)
#         x = self.linear_cnn(x)
    
        x = self.linear_attn(x)
        

#         sequence_lengths = (x > 0).sum(dim=1)
#         sequence_lengths[sequence_lengths == 0.] = 1       
#         x = self.embedding_layer(x)
#         x = x.mean(dim=-2)        
#         lengths_scaling = sequence_lengths.float() / x.size(1)
#         lengths_scaling = lengths_scaling.unsqueeze(1).repeat((1, x.size(-1)))
#         x /= lengths_scaling.to(x.device)        
#         x = self.l1(x)                

#         x = self.relu(x)
#         x = self.l2(x)
#         x = self.relu(x)
#         x = self.lout(x)
        
        return x

CUDA RAM usage 3558456832


In [0]:
model = DeepAverageNetwork(embedding_matrix=vectors, n_classes=n_classes)
model.to(device)

DeepAverageNetwork(
  (embedding_layer): Embedding(220907, 300)
  (l1): Linear(in_features=300, out_features=256, bias=True)
  (lstm): LSTM(300, 64, batch_first=True, bidirectional=True)
  (relu): ReLU()
  (l2): Linear(in_features=256, out_features=128, bias=True)
  (lout): Linear(in_features=128, out_features=28, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (linear): Linear(in_features=128, out_features=28, bias=True)
  (linear_attn): Linear(in_features=156, out_features=28, bias=True)
  (linear_cnn): Linear(in_features=512, out_features=28, bias=True)
  (ak): Linear(in_features=128, out_features=28, bias=True)
  (aq): Linear(in_features=128, out_features=28, bias=True)
  (av): Linear(in_features=128, out_features=28, bias=True)
  (conv1): Conv1d(64, 128, kernel_size=(2,), stride=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(4,), stride=(1,))
  (conv3): Conv1d(64, 128, kernel_size=(6,), stride=(1,))
  (conv4): Conv1d(64, 128, kernel_size=(8,), stride=(1,))
  (mp1): MaxPool1

In [0]:
with torch.no_grad():
    pred = model(x.to('cuda'))

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())

# model = model.to(device)
criterion = criterion.to(device)

In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

In [0]:
epochs = 20
# epochs = 5
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        
        pred = model(x)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y in validation_loader:
        
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():

            pred = model(x)

            pred = pred.cpu()
            y = y.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.3f}, test - {:.3f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.3f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 2:   0%|          | 384/599999 [00:00<04:13, 2362.30it/s, train_loss=1.74]


Losses: train - 1.959, test - 1.535
F1 test - 0.558


Epoch 3:   0%|          | 448/599999 [00:00<03:39, 2725.57it/s, train_loss=1.65]


Losses: train - 1.702, test - 1.445
F1 test - 0.583


Epoch 4:   0%|          | 384/599999 [00:00<03:43, 2684.91it/s, train_loss=1.62]

...


Losses: train - 1.544, test - 1.334
F1 test - 0.613


Epoch 14:   0%|          | 448/599999 [00:00<03:35, 2781.74it/s, train_loss=1.52]


Losses: train - 1.540, test - 1.331
F1 test - 0.614


Epoch 14: 100%|██████████| 599999/599999 [04:00<00:00, 2497.13it/s, train_loss=1.51]



Losses: train - 1.535, test - 1.331
F1 test - 0.613
Early stopping


In [0]:
model.eval()

predictions = []

for x, _ in test_loader:

    x = x.to(device)

    with torch.no_grad():

        pred = model(x)

        pred = pred.cpu()
        
        predictions.append(np.argmax(pred, axis=1))
        
predictions = np.concatenate(predictions).squeeze()

In [0]:
for x, y in test_loader:
  print(y)
  break
    

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [0]:
test['main_category'] = predictions

In [0]:
test

,index,question,main_category
0,0,"Подскажите сайт российский, вещи заказывать: )...",15
1,1,Как заказать Рикардо Милоса в Москве?,15
2,2,"Пустят ли меня в Израиль, если я работаю в ОАЭ?",25
3,3,почему о наличии пальмового масла и консервато...,17
4,4,Вот во всех бедах России винят правительство.....,12
...,...,...,...
199995,199995,Какая девушка по характеру? Что происходит вок...,1
199996,199996,Куда отправиться отдыхать? Дубай или Испания (...,25
199997,199997,парсинг запроса к сайту,14
199998,199998,Можно ли подать заявление на восстановление ср...,4


In [0]:
test = test[['index', 'main_category']]

KeyError: ignored

In [0]:
del test['question']

In [0]:
test.head()

,index,main_category
0,0,15
1,1,15
2,2,25
3,3,17
4,4,12


In [0]:
test.to_csv('gdrive/My Drive/CompLing/submission.csv', index=False)